1.Segmentaion

In [ ]:
import numpy as np                          # For numerical operations like arrays, math, statistics
import tifffile as tiff                     # To read/write hyperspectral TIFF images
from skimage import filters                 # Image processing filters (Laplacian, Gaussian, etc.)
from skimage.filters import threshold_otsu  # Otsu thresholding for image segmentation
from sklearn.cluster import KMeans          # KMeans clustering algorithm
from ipywidgets import interact, IntSlider  # Interactive sliders for Jupyter Notebook
import matplotlib.pyplot as plt             # To plot images and graphs
from matplotlib import cm                   # Colormap utilities
from matplotlib.colors import ListedColormap # For custom colormaps in clustering visualization
from matplotlib.patches import Patch        # To create legend patches for clusters

# Define wavelength range (400–1000 nm, total 204 bands)
num_bands = 204                               # Total number of spectral bands in dataset
wavelength_start = 400                         # Start wavelength in nanometers
wavelength_end = 1000                          # End wavelength in nanometers
wavelengths = np.linspace(wavelength_start, wavelength_end, num_bands)  # Evenly spaced wavelengths for each band

# Function to calculate metrics for a single band
def calculate_band_metrics(band_array):
    # Normalize band values to range 0–1
    band_norm = (band_array - np.nanmin(band_array)) / (np.nanmax(band_array) - np.nanmin(band_array) + 1e-8)

    brightness = np.nanmean(band_norm)                 # Average intensity of the band
    contrast = np.nanstd(band_norm)                    # Contrast as standard deviation
    sharpness = filters.laplace(band_norm).var()      # Sharpness using Laplacian variance

    # High-frequency components for noise estimation
    highpass = band_norm - filters.gaussian(band_norm, sigma=1)  
    noise = np.nanstd(highpass)                        # Standard deviation of high-frequency signal = noise
    signal = np.sqrt(np.nanmean(band_norm**2))        # Signal power
    snr = 20 * np.log10(signal / (noise + 1e-8))      # Signal-to-noise ratio in dB

    # Return all metrics in a dictionary
    return {
        "Brightness": brightness,
        "Contrast": contrast,
        "Sharpness": sharpness,
        "SNR(dB)": snr
    }

# Interactive function for metrics, segmentation, clustering
def show_band_metrics_interactive(img_id, band_idx):
    # Construct file path for the image based on Image ID
    file_path = f"D:/Z Education/University/4-1 Course/Thesis/Hyper leaf dataset/image/{img_id:05d}.tiff"

    try:
        cube = tiff.imread(file_path)            # Load hyperspectral cube (bands, rows, cols)
        bands, rows, cols = cube.shape           # Get cube dimensions

        # Print basic information about the image
        print(f"Loaded Image ID: {img_id}")
        print(f"Image width: {cols}, height: {rows}, bands: {bands}, dtype: {cube.dtype}")
        bit_depth = cube.dtype.itemsize * 8      # Calculate bit depth from data type
        print(f"Bit depth: {bit_depth}\n")

        cube = np.transpose(cube, (1, 2, 0))    # Rearrange cube to (rows, cols, bands) for visualization
        band_array = cube[:, :, band_idx]        # Extract the selected band

        metrics = calculate_band_metrics(band_array)  # Compute metrics for the band

        # Print band wavelength if available
        if len(wavelengths) == bands:
            band_wavelength = wavelengths[band_idx]
            print(f"Selected Band: {band_idx} | Wavelength: {band_wavelength:.2f} nm")
        else:
            print(f"Selected Band: {band_idx} | Wavelength info not available")

        # Print metrics
        print(f"Brightness: {metrics['Brightness']:.4f}")
        print(f"Contrast: {metrics['Contrast']:.4f}")
        print(f"Sharpness: {metrics['Sharpness']:.4f}")
        print(f"SNR(dB): {metrics['SNR(dB)']:.4f}")

        # Leaf segmentation using Otsu threshold
        thresh = threshold_otsu(band_array)      # Compute Otsu threshold
        mask = band_array > thresh                # Create binary mask of leaf area
        segmented_leaf = band_array * mask        # Apply mask to band for segmented leaf

        # KMeans clustering for disease detection
        n_clusters = 3
        if np.sum(mask) > 10:                     # Ensure there are enough pixels to cluster
            leaf_pixels_band = band_array[mask].reshape(-1, 1)  # Flatten leaf pixels
            kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10).fit(leaf_pixels_band)  # Run KMeans
            disease_mask = np.zeros_like(band_array)             # Initialize cluster mask
            disease_mask[mask] = kmeans.labels_ + 1              # Assign cluster labels to leaf pixels
        else:
            disease_mask = np.zeros_like(band_array)            # If not enough pixels, empty mask
            kmeans = None

        # Extract leaf pixels for spectral analysis
        leaf_pixels_cube = cube[mask, :]          # Extract all bands for leaf pixels
        mean_spectrum = np.mean(leaf_pixels_cube, axis=0)  # Mean reflectance spectrum
        min_spectrum = np.min(leaf_pixels_cube, axis=0)    # Minimum reflectance
        std_spectrum = np.std(leaf_pixels_cube, axis=0)    # Standard deviation

        # Visualization (2x2 layout)
        fig, axes = plt.subplots(2, 2, figsize=(12, 4))  
        axes = axes.ravel()                         # Flatten axes array

        # RGB Composite creation using approximate R,G,B wavelengths
        def find_band_index(target_wl):
            return np.argmin(np.abs(wavelengths - target_wl))  # Find closest band to target wavelength

        r_idx = find_band_index(660)                 # Red band (~660 nm)
        g_idx = find_band_index(550)                 # Green band (~550 nm)
        b_idx = find_band_index(470)                 # Blue band (~470 nm)

        rgb_img = np.stack([cube[:, :, r_idx],
                            cube[:, :, g_idx],
                            cube[:, :, b_idx]], axis=-1)   # Combine 3 bands to RGB
        rgb_img = (rgb_img - np.min(rgb_img)) / (np.max(rgb_img) - np.min(rgb_img) + 1e-8)  # Normalize

        axes[0].imshow(rgb_img)                     # Show RGB image
        axes[0].set_title(f"Original Leaf ({img_id})")
        axes[0].axis("off")

        axes[1].imshow(mask, cmap='gray')           # Show Otsu binary mask
        axes[1].set_title(f"Otsu Binary Mask (Band {band_idx})", fontsize=12)
        axes[1].axis("off")

        axes[2].imshow(segmented_leaf, cmap='gray') # Show segmented leaf
        axes[2].set_title("Segmented Leaf (Grayscale)", fontsize=12)
        axes[2].axis("off")

        cluster_cmap = ListedColormap(["black", "green", "orange", "blue"])  # Cluster colormap
        axes[3].imshow(disease_mask, cmap=cluster_cmap, vmin=0, vmax=3)     # Show clusters
        axes[3].set_title("Leaf Clusters (Healthy, Mild, Severe)", fontsize=12)
        axes[3].axis("off")

        legend_elements = [Patch(facecolor="green", edgecolor='k', label="Healthy"),
                           Patch(facecolor="orange", edgecolor='k', label="Mild Disease"),
                           Patch(facecolor="blue", edgecolor='k', label="Severe Disease")]
        axes[3].legend(handles=legend_elements, loc='upper right', fontsize=8)  # Add legend

        plt.tight_layout()
        plt.show()

        # Summary spectrum plot
        plt.figure(figsize=(9, 6))
        plt.plot(wavelengths, mean_spectrum, '-g', label="Mean Reflectance")   # Mean spectrum
        plt.plot(wavelengths, min_spectrum, '-r', label="Minimum Reflectance") # Min spectrum
        plt.plot(wavelengths, std_spectrum, '-b', label="Std Deviation")       # Std deviation

        # Highlight minimum reflectance point
        min_val = np.min(min_spectrum)
        min_idx = np.argmin(min_spectrum)
        min_wl = wavelengths[min_idx]
        plt.scatter(min_wl, min_val, color='red', zorder=5)
        plt.text(min_wl + 5, min_val, f"Min = {min_val:.4f} @ {min_wl:.1f} nm", color='red', fontsize=10, va='bottom')

        plt.xlabel("Wavelength (nm)")
        plt.ylabel("Reflectance")
        plt.title(f"Summary Spectrum (Image ID {img_id})")
        plt.grid(True)
        plt.legend(loc="center left", bbox_to_anchor=(1, 0.5))
        plt.tight_layout()
        plt.show()

        # Cluster-wise spectra
        if kmeans is not None:
            cluster_names = ["Healthy", "Mild Disease", "Severe Disease"]
            cluster_colors = ["green", "orange", "blue"]

            plt.figure(figsize=(9, 6))
            for cluster_id in range(n_clusters):
                cluster_mask_pixels = (disease_mask == (cluster_id + 1))          # Mask for cluster
                if np.sum(cluster_mask_pixels) > 0:
                    cluster_pixels_cube = cube[cluster_mask_pixels, :]            # Extract cluster pixels
                    cluster_mean = np.mean(cluster_pixels_cube, axis=0)           # Mean spectrum
                    plt.plot(wavelengths, cluster_mean, label=f"{cluster_names[cluster_id]} Spectrum",
                             color=cluster_colors[cluster_id])
            plt.xlabel("Wavelength (nm)")
            plt.ylabel("Reflectance")
            plt.title(f"Cluster-wise Spectra (Image ID {img_id})")
            plt.grid(True)
            plt.legend()
            plt.show()

        # Individual ROI pixel spectra
        num_pixels = leaf_pixels_cube.shape[0]          # Number of pixels in ROI
        fig, ax = plt.subplots(figsize=(9, 6))
        for i in range(num_pixels):
            ax.plot(wavelengths, leaf_pixels_cube[i], color=cm.viridis(i / num_pixels), linewidth=1)

        # Add colorbar showing pixel index
        sm = plt.cm.ScalarMappable(cmap=cm.viridis, norm=plt.Normalize(vmin=0, vmax=num_pixels - 1))
        sm.set_array([])
        cbar = plt.colorbar(sm, ax=ax)
        cbar.set_label("Pixel Index (ROI)")

        ax.set_xlabel("Wavelength (nm)")
        ax.set_ylabel("Reflectance")
        ax.set_title(f"Spectral Curves of ROI Pixels (Image ID {img_id})")
        plt.show()

    except FileNotFoundError:
        print(f"File for Image ID {img_id} not found.")  # Handle missing file error

# Dataset info
min_id = 0
max_id = 2409
first_img_file = f"D:/Z Education/University/4-1 Course/Thesis/Hyper leaf dataset/image/{min_id:05d}.tiff"
first_cube = tiff.imread(first_img_file)       # Load first image to get number of bands
total_bands = first_cube.shape[0]

# Interactive sliders for Image ID and Band selection
interact(
    show_band_metrics_interactive,
    img_id=IntSlider(min=min_id, max=max_id, step=1, value=min_id, description='Image ID'),
    band_idx=IntSlider(min=0, max=total_bands-1, step=1, value=0, description='Band')
)


In [ ]:
import numpy as np                          # For numerical operations like arrays, math, statistics
import tifffile as tiff                     # To read/write hyperspectral TIFF images
from skimage import filters                 # Image processing filters (Laplacian, Gaussian, etc.)
from skimage.filters import threshold_otsu  # Otsu thresholding for image segmentation
from sklearn.cluster import KMeans          # KMeans clustering algorithm
from ipywidgets import interact, IntSlider  # Interactive sliders for Jupyter Notebook
import matplotlib.pyplot as plt             # To plot images and graphs
from matplotlib import cm                   # Colormap utilities
from matplotlib.colors import ListedColormap # For custom colormaps in clustering visualization
from matplotlib.patches import Patch        # To create legend patches for clusters

# Define wavelength range (400–1000 nm, total 204 bands)
num_bands = 204                               # Total number of spectral bands in dataset
wavelength_start = 400                         # Start wavelength in nanometers
wavelength_end = 1000                          # End wavelength in nanometers
wavelengths = np.linspace(wavelength_start, wavelength_end, num_bands)  # Evenly spaced wavelengths for each band

# Function to calculate metrics for a single band
def calculate_band_metrics(band_array):
    # Normalize band values to range 0–1
    band_norm = (band_array - np.nanmin(band_array)) / (np.nanmax(band_array) - np.nanmin(band_array) + 1e-8)

    brightness = np.nanmean(band_norm)                 # Average intensity of the band
    contrast = np.nanstd(band_norm)                    # Contrast as standard deviation
    sharpness = filters.laplace(band_norm).var()      # Sharpness using Laplacian variance

    # High-frequency components for noise estimation
    highpass = band_norm - filters.gaussian(band_norm, sigma=1)  
    noise = np.nanstd(highpass)                        # Standard deviation of high-frequency signal = noise
    signal = np.sqrt(np.nanmean(band_norm**2))        # Signal power
    snr = 20 * np.log10(signal / (noise + 1e-8))      # Signal-to-noise ratio in dB

    # Return all metrics in a dictionary
    return {
        "Brightness": brightness,
        "Contrast": contrast,
        "Sharpness": sharpness,
        "SNR(dB)": snr
    }

# Interactive function for metrics, segmentation, clustering
def show_band_metrics_interactive(img_id, band_idx):
    # Construct file path for the image based on Image ID
    file_path = f"D:/Z Education/University/4-1 Course/Thesis/Hyper leaf dataset/image/{img_id:05d}.tiff"

    try:
        cube = tiff.imread(file_path)            # Load hyperspectral cube (bands, rows, cols)
        bands, rows, cols = cube.shape           # Get cube dimensions

        # Print basic information about the image
        print(f"Loaded Image ID: {img_id}")
        print(f"Image width: {cols}, height: {rows}, bands: {bands}, dtype: {cube.dtype}")
        bit_depth = cube.dtype.itemsize * 8      # Calculate bit depth from data type
        print(f"Bit depth: {bit_depth}\n")

        cube = np.transpose(cube, (1, 2, 0))    # Rearrange cube to (rows, cols, bands) for visualization
        band_array = cube[:, :, band_idx]        # Extract the selected band

        metrics = calculate_band_metrics(band_array)  # Compute metrics for the band

        # Print band wavelength if available
        if len(wavelengths) == bands:
            band_wavelength = wavelengths[band_idx]
            print(f"Selected Band: {band_idx} | Wavelength: {band_wavelength:.2f} nm")
        else:
            band_wavelength = None
            print(f"Selected Band: {band_idx} | Wavelength info not available")

        # Print metrics
        print(f"Brightness: {metrics['Brightness']:.4f}")
        print(f"Contrast: {metrics['Contrast']:.4f}")
        print(f"Sharpness: {metrics['Sharpness']:.4f}")
        print(f"SNR(dB): {metrics['SNR(dB)']:.4f}")

        # Leaf segmentation using Otsu threshold
        thresh = threshold_otsu(band_array)      # Compute Otsu threshold
        mask = band_array > thresh                # Create binary mask of leaf area
        segmented_leaf = band_array * mask        # Apply mask to band for segmented leaf

        # ✅ Show Selected Band for Segmentation (New Figure)
        plt.figure(figsize=(6, 5))
        plt.imshow(band_array, cmap='gray')
        if band_wavelength is not None:
            plt.title(f"Selected Band for Segmentation (Band {band_idx} | {band_wavelength:.2f} nm)", fontsize=12)
        else:
            plt.title(f"Selected Band for Segmentation (Band {band_idx})", fontsize=12)
        plt.axis("off")
       
        plt.tight_layout()
        plt.show()

        # KMeans clustering for disease detection
        n_clusters = 3
        if np.sum(mask) > 10:                     # Ensure there are enough pixels to cluster
            leaf_pixels_band = band_array[mask].reshape(-1, 1)  # Flatten leaf pixels
            kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10).fit(leaf_pixels_band)  # Run KMeans
            disease_mask = np.zeros_like(band_array)             # Initialize cluster mask
            disease_mask[mask] = kmeans.labels_ + 1              # Assign cluster labels to leaf pixels
        else:
            disease_mask = np.zeros_like(band_array)            # If not enough pixels, empty mask
            kmeans = None

        # Extract leaf pixels for spectral analysis
        leaf_pixels_cube = cube[mask, :]          # Extract all bands for leaf pixels
        mean_spectrum = np.mean(leaf_pixels_cube, axis=0)  # Mean reflectance spectrum
        min_spectrum = np.min(leaf_pixels_cube, axis=0)    # Minimum reflectance
        std_spectrum = np.std(leaf_pixels_cube, axis=0)    # Standard deviation

        # Visualization (2x2 layout)
        fig, axes = plt.subplots(2, 2, figsize=(12, 4))  
        axes = axes.ravel()                         # Flatten axes array

        # RGB Composite creation using approximate R,G,B wavelengths
        def find_band_index(target_wl):
            return np.argmin(np.abs(wavelengths - target_wl))  # Find closest band to target wavelength

        r_idx = find_band_index(660)                 # Red band (~660 nm)
        g_idx = find_band_index(550)                 # Green band (~550 nm)
        b_idx = find_band_index(470)                 # Blue band (~470 nm)

        rgb_img = np.stack([cube[:, :, r_idx],
                            cube[:, :, g_idx],
                            cube[:, :, b_idx]], axis=-1)   # Combine 3 bands to RGB
        rgb_img = (rgb_img - np.min(rgb_img)) / (np.max(rgb_img) - np.min(rgb_img) + 1e-8)  # Normalize

        axes[0].imshow(rgb_img)                     # Show RGB image
        axes[0].set_title(f"Original Leaf (Image ID {img_id})")
        axes[0].axis("off")

        axes[1].imshow(mask, cmap='gray')           # Show Otsu binary mask
        axes[1].set_title(f"Otsu Binary Mask (Band {band_idx})", fontsize=12)
        axes[1].axis("off")

        axes[2].imshow(segmented_leaf, cmap='gray') # Show segmented leaf
        axes[2].set_title("Segmented Leaf (Grayscale)", fontsize=12)
        axes[2].axis("off")

        cluster_cmap = ListedColormap(["black", "green", "orange", "blue"])  # Cluster colormap
        axes[3].imshow(disease_mask, cmap=cluster_cmap, vmin=0, vmax=3)     # Show clusters
        axes[3].set_title("Leaf Clusters (Healthy, Mild, Severe)", fontsize=12)
        axes[3].axis("off")

        legend_elements = [Patch(facecolor="green", edgecolor='k', label="Healthy"),
                           Patch(facecolor="orange", edgecolor='k', label="Mild Disease"),
                           Patch(facecolor="blue", edgecolor='k', label="Severe Disease")]
        axes[3].legend(handles=legend_elements, loc='upper right', fontsize=8)  # Add legend

        plt.tight_layout()
        plt.show()

        # Summary spectrum plot
        plt.figure(figsize=(9, 6))
        plt.plot(wavelengths, mean_spectrum, '-g', label="Mean Reflectance")   # Mean spectrum
        plt.plot(wavelengths, min_spectrum, '-r', label="Minimum Reflectance") # Min spectrum
        plt.plot(wavelengths, std_spectrum, '-b', label="Std Deviation")       # Std deviation

        # Highlight minimum reflectance point
        min_val = np.min(min_spectrum)
        min_idx = np.argmin(min_spectrum)
        min_wl = wavelengths[min_idx]
        plt.scatter(min_wl, min_val, color='red', zorder=5)
        plt.text(min_wl + 5, min_val, f"Min = {min_val:.4f} @ {min_wl:.1f} nm", color='red', fontsize=10, va='bottom')

        plt.xlabel("Wavelength (nm)")
        plt.ylabel("Reflectance")
        plt.title(f"Summary Spectrum (Image ID {img_id})")
        plt.grid(True)
        plt.legend(loc="center left", bbox_to_anchor=(1, 0.5))
        plt.tight_layout()
        plt.show()

        # Cluster-wise spectra
        if kmeans is not None:
            cluster_names = ["Healthy", "Mild Disease", "Severe Disease"]
            cluster_colors = ["green", "orange", "blue"]

            plt.figure(figsize=(9, 6))
            for cluster_id in range(n_clusters):
                cluster_mask_pixels = (disease_mask == (cluster_id + 1))          # Mask for cluster
                if np.sum(cluster_mask_pixels) > 0:
                    cluster_pixels_cube = cube[cluster_mask_pixels, :]            # Extract cluster pixels
                    cluster_mean = np.mean(cluster_pixels_cube, axis=0)           # Mean spectrum
                    plt.plot(wavelengths, cluster_mean, label=f"{cluster_names[cluster_id]} Spectrum",
                             color=cluster_colors[cluster_id])
            plt.xlabel("Wavelength (nm)")
            plt.ylabel("Reflectance")
            plt.title(f"Cluster-wise Spectra (Image ID {img_id})")
            plt.grid(True)
            plt.legend()
            plt.show()

        # Individual ROI pixel spectra

# Individual ROI pixel spectra
            # === Original Spectra for Multiple Leaves ===
       # === Original Spectra for Specific Range of Leaves ===
        csv_path = r"D:/Z Education/University/4-1 Course/Thesis/Hyper leaf dataset/normalized_minmax_with_target_2.csv"
        df = pd.read_csv(csv_path)

        # Extract wavelength columns (all except 'Yield')
        wavelength_cols = [col for col in df.columns if col != 'Yield']
        wavelengths_csv = np.array([float(col.replace(' nm','')) for col in wavelength_cols])

        # Select a specific range of rows (e.g., index 450 to 650)
        subset_df = df.iloc[450:650]  
        spectra = subset_df[wavelength_cols].values
        yields = subset_df['Yield'].values

        norm = plt.Normalize(vmin=yields.min(), vmax=yields.max())
        cmap = plt.cm.viridis
        colors = cmap(norm(yields))

        fig, ax = plt.subplots(figsize=(10,6))
        for i in range(len(spectra)):
            ax.plot(wavelengths_csv, spectra[i], color=colors[i], linewidth=1.5)

        ax.set_xlabel("Wavelength (nm)", fontweight='bold')
        ax.set_ylabel("Reflectance", fontweight='bold')
        ax.set_title("Original Spectra of Leaves (Color = Yield)", fontweight='bold')

        ax.tick_params(axis='both', which='major', labelsize=10, width=1.2)
        for label in ax.get_xticklabels() + ax.get_yticklabels():
            label.set_fontweight('bold')

        sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])
        cbar = plt.colorbar(sm, ax=ax)
        cbar.set_label("Yield Value", fontweight='bold')
        for label in cbar.ax.get_yticklabels():
            label.set_fontweight('bold')

        plt.tight_layout()
        plt.show()


        # Add colorbar showing pixel index (subset)
        sm = plt.cm.ScalarMappable(cmap=cm.viridis, norm=plt.Normalize(vmin=0, vmax=len(num_pixels) - 1))
        sm.set_array([])
        cbar = plt.colorbar(sm, ax=ax)
        cbar.set_label("Pixel Index (Subset of ROI)", fontweight='bold')  # ✅ Bold colorbar label

        # Bold axis labels and title
        ax.set_xlabel("Wavelength (nm)", fontweight='bold')
        ax.set_ylabel("Reflectance", fontweight='bold')
        ax.set_title("Original Spectra", fontweight='bold')

        # Bold tick labels
        ax.tick_params(axis='both', which='major', labelsize=10, width=1.2, labelcolor='black')
        for label in ax.get_xticklabels() + ax.get_yticklabels():
            label.set_fontweight('bold')

        # Bold colorbar tick labels
        for label in cbar.ax.get_yticklabels():
            label.set_fontweight('bold')

        plt.show()
            
        # Add colorbar showing pixel index
        sm = plt.cm.ScalarMappable(cmap=cm.viridis, norm=plt.Normalize(vmin=0, vmax=num_pixels - 1))
        sm.set_array([])
        cbar = plt.colorbar(sm, ax=ax)
        cbar.set_label("Pixel Index (ROI)")

        ax.set_xlabel("Wavelength (nm)")
        ax.set_ylabel("Reflectance")
        ax.set_title(f"Spectral Curves of ROI Pixels (Image ID {img_id})")
        plt.show()

    except FileNotFoundError:
        print(f"File for Image ID {img_id} not found.")  # Handle missing file error

# Dataset info
min_id = 0
max_id = 2409
first_img_file = f"D:/Z Education/University/4-1 Course/Thesis/Hyper leaf dataset/image/{min_id:05d}.tiff"
first_cube = tiff.imread(first_img_file)       # Load first image to get number of bands
total_bands = first_cube.shape[0]

# Interactive sliders for Image ID and Band selection
interact(
    show_band_metrics_interactive,
    img_id=IntSlider(min=min_id, max=max_id, step=1, value=min_id, description='Image ID'),
    band_idx=IntSlider(min=0, max=total_bands-1, step=1, value=0, description='Band')
)


In [ ]:
import numpy as np
import tifffile as tiff
from skimage import filters
from skimage.filters import threshold_otsu
from sklearn.cluster import KMeans
from ipywidgets import interact, IntSlider
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch

# Define wavelength range (400–1000 nm, total 204 bands)
num_bands = 204
wavelength_start = 400
wavelength_end = 1000
wavelengths = np.linspace(wavelength_start, wavelength_end, num_bands)

# Function to calculate metrics for a single band
def calculate_band_metrics(band_array):
    band_norm = (band_array - np.nanmin(band_array)) / (np.nanmax(band_array) - np.nanmin(band_array) + 1e-8)
    brightness = np.nanmean(band_norm)
    contrast = np.nanstd(band_norm)
    sharpness = filters.laplace(band_norm).var()
    highpass = band_norm - filters.gaussian(band_norm, sigma=1)
    noise = np.nanstd(highpass)
    signal = np.sqrt(np.nanmean(band_norm**2))
    snr = 20 * np.log10(signal / (noise + 1e-8))
    return {"Brightness": brightness, "Contrast": contrast, "Sharpness": sharpness, "SNR(dB)": snr}

# Interactive function for metrics, segmentation, clustering
def show_band_metrics_interactive(img_id, band_idx):
    file_path = f"D:/Z Education/University/4-1 Course/Thesis/Hyper leaf dataset/image/{img_id:05d}.tiff"
    try:
        cube = tiff.imread(file_path)
        bands, rows, cols = cube.shape

        print(f"Loaded Image ID: {img_id}")
        print(f"Image width: {cols}, height: {rows}, bands: {bands}, dtype: {cube.dtype}")
        print(f"Bit depth: {cube.dtype.itemsize * 8}\n")

        cube = np.transpose(cube, (1, 2, 0))
        band_array = cube[:, :, band_idx]
        metrics = calculate_band_metrics(band_array)

        band_wavelength = wavelengths[band_idx] if len(wavelengths) == bands else None
        print(f"Selected Band: {band_idx} | Wavelength: {band_wavelength if band_wavelength else 'N/A'} nm")
        print(f"Brightness: {metrics['Brightness']:.4f}")
        print(f"Contrast: {metrics['Contrast']:.4f}")
        print(f"Sharpness: {metrics['Sharpness']:.4f}")
        print(f"SNR(dB): {metrics['SNR(dB)']:.4f}")

        # Leaf segmentation using Otsu threshold
        thresh = threshold_otsu(band_array)
        mask = band_array > thresh
        segmented_leaf = band_array * mask

        # Show Selected Band
        plt.figure(figsize=(6,5))
        plt.imshow(band_array, cmap='gray')
        plt.title(f"Selected Band (Band {band_idx} | {band_wavelength:.2f} nm)" if band_wavelength else f"Selected Band {band_idx}", fontsize=12, fontweight='bold')
        plt.axis("off")
        plt.tight_layout()
        plt.show()

        # KMeans clustering for disease detection
        n_clusters = 3
        disease_mask = np.zeros_like(band_array)
        kmeans = None
        if np.sum(mask) > 10:
            leaf_pixels_band = band_array[mask].reshape(-1, 1)
            kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10).fit(leaf_pixels_band)
            disease_mask[mask] = kmeans.labels_ + 1

        # Extract leaf pixels for spectral analysis
        leaf_pixels_cube = cube[mask, :]
        mean_spectrum = np.mean(leaf_pixels_cube, axis=0)
        min_spectrum = np.min(leaf_pixels_cube, axis=0)
        std_spectrum = np.std(leaf_pixels_cube, axis=0)

        # Visualization (RGB, Mask, Segmented, Clusters)
        fig, axes = plt.subplots(2, 2, figsize=(12,4))
        axes = axes.ravel()
        def find_band_index(target_wl): return np.argmin(np.abs(wavelengths - target_wl))
        rgb_img = np.stack([cube[:, :, find_band_index(660)],
                            cube[:, :, find_band_index(550)],
                            cube[:, :, find_band_index(470)]], axis=-1)
        rgb_img = (rgb_img - np.min(rgb_img)) / (np.max(rgb_img) - np.min(rgb_img) + 1e-8)
        axes[0].imshow(rgb_img)
        axes[0].set_title(f"Original Leaf (Image ID {img_id})", fontweight='bold')
        axes[0].axis("off")

        axes[1].imshow(mask, cmap='gray')
        axes[1].set_title(f"Otsu Binary Mask (Band {band_idx})", fontweight='bold')
        axes[1].axis("off")

        axes[2].imshow(segmented_leaf, cmap='gray')
        axes[2].set_title("Segmented Leaf (Grayscale)", fontweight='bold')
        axes[2].axis("off")

        cluster_cmap = ListedColormap(["black", "green", "orange", "blue"])
        axes[3].imshow(disease_mask, cmap=cluster_cmap, vmin=0, vmax=3)
        axes[3].set_title("Leaf Clusters (Healthy, Mild, Severe)", fontweight='bold')
        axes[3].axis("off")

        legend_elements = [Patch(facecolor="green", edgecolor='k', label="Healthy"),
                           Patch(facecolor="orange", edgecolor='k', label="Mild Disease"),
                           Patch(facecolor="blue", edgecolor='k', label="Severe Disease")]
        axes[3].legend(handles=legend_elements, loc='upper right', fontsize=8)

        plt.tight_layout()
        plt.show()

        # Summary Spectrum
        plt.figure(figsize=(9,6))
        plt.plot(wavelengths, mean_spectrum, '-g', label="Mean Reflectance")
        plt.plot(wavelengths, min_spectrum, '-r', label="Minimum Reflectance")
        plt.plot(wavelengths, std_spectrum, '-b', label="Std Deviation")
        min_val = np.min(min_spectrum)
        min_idx = np.argmin(min_spectrum)
        min_wl = wavelengths[min_idx]
        plt.scatter(min_wl, min_val, color='red', zorder=5)
        plt.text(min_wl+5, min_val, f"Min={min_val:.4f}@{min_wl:.1f}nm", color='red', fontsize=10, va='bottom')
        plt.xlabel("Wavelength (nm)", fontweight='bold')
        plt.ylabel("Reflectance", fontweight='bold')
        plt.title(f"Summary Spectrum (Image ID {img_id})", fontweight='bold')
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()

        # Cluster-wise Spectra
        if kmeans is not None:
            cluster_names = ["Healthy", "Mild Disease", "Severe Disease"]
            cluster_colors = ["green", "orange", "blue"]
            plt.figure(figsize=(9,6))
            for cluster_id in range(n_clusters):
                cluster_mask_pixels = (disease_mask == (cluster_id + 1))
                if np.sum(cluster_mask_pixels) > 0:
                    cluster_pixels_cube = cube[cluster_mask_pixels, :]
                    cluster_mean = np.mean(cluster_pixels_cube, axis=0)
                    plt.plot(wavelengths, cluster_mean, label=f"{cluster_names[cluster_id]} Spectrum", color=cluster_colors[cluster_id])
            plt.xlabel("Wavelength (nm)", fontweight='bold')
            plt.ylabel("Reflectance", fontweight='bold')
            plt.title(f"Cluster-wise Spectra (Image ID {img_id})", fontweight='bold')
            plt.grid(True)
            plt.legend()
            plt.tight_layout()
            plt.show()

        # Original Spectra for Specific Range of Leaves
        csv_path = r"D:/Z Education/University/4-1 Course/Thesis/Hyper leaf dataset/normalized_minmax_with_target_2.csv"
        df = pd.read_csv(csv_path)
        wavelength_cols = [col for col in df.columns if col != 'Yield']
        wavelengths_csv = np.array([float(col.replace(' nm','')) for col in wavelength_cols])
        subset_df = df.iloc[20:70]
        spectra = subset_df[wavelength_cols].values
        yields = subset_df['Yield'].values
        norm = plt.Normalize(vmin=yields.min(), vmax=yields.max())
        cmap = plt.cm.viridis
        colors = cmap(norm(yields))
        fig, ax = plt.subplots(figsize=(10,6))
        for i in range(len(spectra)):
            ax.plot(wavelengths_csv, spectra[i], color=colors[i], linewidth=1.5)
        ax.set_xlabel("Wavelength (nm)", fontweight='bold')
        ax.set_ylabel("Reflectance", fontweight='bold')
        ax.set_title("Original Spectra ", fontweight='bold')
        ax.tick_params(axis='both', which='major', labelsize=10, width=1.2)
        for label in ax.get_xticklabels() + ax.get_yticklabels():
            label.set_fontweight('bold')
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])
        cbar = plt.colorbar(sm, ax=ax)
        cbar.set_label("Yield Value", fontweight='bold')
        for label in cbar.ax.get_yticklabels():
            label.set_fontweight('bold')
        plt.tight_layout()
        plt.show()

    except FileNotFoundError:
        print(f"File for Image ID {img_id} not found.")

# Dataset info
min_id = 0
max_id = 2409
first_cube = tiff.imread(f"D:/Z Education/University/4-1 Course/Thesis/Hyper leaf dataset/image/{min_id:05d}.tiff")
total_bands = first_cube.shape[0]

# Interactive sliders
interact(
    show_band_metrics_interactive,
    img_id=IntSlider(min=min_id, max=max_id, step=1, value=min_id, description='Image ID'),
    band_idx=IntSlider(min=0, max=total_bands-1, step=1, value=0, description='Band')
)


1.1 Prediction map

In [ ]:
# Interactive Yield Prediction Map Visualization (Scaled by dataset yield)

import numpy as np
import pandas as pd
import tifffile as tiff
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider
from matplotlib import cm

#  CSV Load 
csv_path = r"D:/Z Education/University/4-1 Course/Thesis/Hyper leaf dataset/normalized_minmax_with_target_2.csv"
data = pd.read_csv(csv_path)

# Extract wavelength band columns (like '400nm', '402nm', etc.)
band_columns = [col for col in data.columns if 'nm' in col.lower()]
wavelengths = np.array([float(col.lower().replace('nm', '')) for col in band_columns])

# Try to get Yield column if exists
yield_col = None
for c in data.columns:
    if 'yield' in c.lower():
        yield_col = c
        break

# Get dataset min and max yield
if yield_col:
    yield_min = data[yield_col].min()
    yield_max = data[yield_col].max()
else:
    yield_min = 0
    yield_max = 1  # fallback

#  Visualization Function 
def show_yield_visualization(img_id, band_idx):
    file_path = f"D:/Z Education/University/4-1 Course/Thesis/Hyper leaf dataset/image/{img_id:05d}.tiff"
    
    try:
        # Load hyperspectral image (Bands × Rows × Cols)
        cube = tiff.imread(file_path)
        bands, rows, cols = cube.shape
        cube = np.transpose(cube, (1, 2, 0))  # Rows × Cols × Bands

        # RGB Composite (approx. 660nm, 550nm, 470nm)
        def find_band_index(target_wl):
            return np.argmin(np.abs(wavelengths - target_wl))

        r_idx = find_band_index(660)
        g_idx = find_band_index(550)
        b_idx = find_band_index(470)

        rgb_img = np.stack([
            (cube[:, :, r_idx] - np.min(cube[:, :, r_idx])) / (np.max(cube[:, :, r_idx]) - np.min(cube[:, :, r_idx]) + 1e-8),
            (cube[:, :, g_idx] - np.min(cube[:, :, g_idx])) / (np.max(cube[:, :, g_idx]) - np.min(cube[:, :, g_idx]) + 1e-8),
            (cube[:, :, b_idx] - np.min(cube[:, :, b_idx])) / (np.max(cube[:, :, b_idx]) - np.min(cube[:, :, b_idx]) + 1e-8)
        ], axis=-1)

        # Prediction Map (selected band)
        prediction_map = cube[:, :, band_idx]
        prediction_map = (prediction_map - np.min(prediction_map)) / (np.max(prediction_map) - np.min(prediction_map) + 1e-8)
        prediction_map = prediction_map * (yield_max - yield_min) + yield_min

        #  Actual yield for this image 
        if yield_col:
            actual_yield = data[yield_col].iloc[img_id]  # Use row index as image ID
        else:
            actual_yield = np.nan

        #  Plot 
        fig, axes = plt.subplots(2, 1, figsize=(7, 5))

        axes[0].imshow(rgb_img)
        axes[0].set_title(f"RGB Image (ID: {img_id})", fontsize=12, fontweight='bold')
        axes[0].axis("off")

        im = axes[1].imshow(prediction_map, cmap='jet', vmin=yield_min, vmax=yield_max)
        axes[1].set_title(f"Prediction Map (Band {band_idx})", fontsize=12, fontweight='bold')
        axes[1].axis("off")

        # Only change: show actual yield value in text
        axes[1].text(0.5, -0.05, f"Yield: {actual_yield:.2f} kg",
                     ha='center', va='top', fontsize=12, fontweight='bold',
                     transform=axes[1].transAxes)

        cbar = fig.colorbar(im, ax=axes[1], orientation='vertical', fraction=0.046, pad=0.04)
        cbar.set_label('Yield (kg)', fontsize=11, fontweight='bold')
        for t in cbar.ax.get_yticklabels():
            t.set_fontweight('bold')

        plt.tight_layout(rect=[0, 0, 1, 1])
        plt.show()

    except FileNotFoundError:
        print(f" File not found for Image ID {img_id}")
    except Exception as e:
        print(f"Error for Image ID {img_id}: {e}")

#  Interactive Widget 
interact(show_yield_visualization,
         img_id=IntSlider(min=0, max=len(data)-1, step=1, value=0),
         band_idx=IntSlider(min=0, max=len(band_columns)-1, step=1, value=0))


In [ ]:
# ==============================================================
# Interactive Yield Prediction Map Visualization (Side-by-side view)
# ==============================================================

import numpy as np
import pandas as pd
import tifffile as tiff
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

#  CSV Load 
csv_path = r"D:/Z Education/University/4-1 Course/Thesis/Hyper leaf dataset/normalized_minmax_with_target_2.csv"
data = pd.read_csv(csv_path)

# Extract wavelength band columns (like '400nm', '402nm', etc.)
band_columns = [col for col in data.columns if 'nm' in col.lower()]
wavelengths = np.array([float(col.lower().replace('nm', '')) for col in band_columns])

# Try to get Yield column if exists
yield_col = None
for c in data.columns:
    if 'yield' in c.lower():
        yield_col = c
        break

# Get dataset min and max yield
if yield_col:
    yield_min = data[yield_col].min()
    yield_max = data[yield_col].max()
else:
    yield_min = 0
    yield_max = 1  # fallback

# ==============================================================
# Visualization Function
# ==============================================================
def show_yield_visualization(img_id, band_idx):
    file_path = f"D:/Z Education/University/4-1 Course/Thesis/Hyper leaf dataset/image/{img_id:05d}.tiff"
    
    try:
        # Load hyperspectral image (Bands × Rows × Cols)
        cube = tiff.imread(file_path)
        bands, rows, cols = cube.shape
        cube = np.transpose(cube, (1, 2, 0))  # Rows × Cols × Bands

        # RGB Composite (approx. 660nm, 550nm, 470nm)
        def find_band_index(target_wl):
            return np.argmin(np.abs(wavelengths - target_wl))

        r_idx = find_band_index(660)
        g_idx = find_band_index(550)
        b_idx = find_band_index(470)

        rgb_img = np.stack([
            (cube[:, :, r_idx] - np.min(cube[:, :, r_idx])) / (np.max(cube[:, :, r_idx]) - np.min(cube[:, :, r_idx]) + 1e-8),
            (cube[:, :, g_idx] - np.min(cube[:, :, g_idx])) / (np.max(cube[:, :, g_idx]) - np.min(cube[:, :, g_idx]) + 1e-8),
            (cube[:, :, b_idx] - np.min(cube[:, :, b_idx])) / (np.max(cube[:, :, b_idx]) - np.min(cube[:, :, b_idx]) + 1e-8)
        ], axis=-1)

        # Prediction Map (selected band)
        prediction_map = cube[:, :, band_idx]
        prediction_map = (prediction_map - np.min(prediction_map)) / (np.max(prediction_map) - np.min(prediction_map) + 1e-8)
        prediction_map = prediction_map * (yield_max - yield_min) + yield_min

        # Actual yield for this image
        if yield_col:
            actual_yield = data[yield_col].iloc[img_id]
        else:
            actual_yield = np.nan

        # ==============================================================
        # Plot (Side-by-side: RGB Left | Prediction Right)
        # ==============================================================
        fig, axes = plt.subplots(1, 2, figsize=(15, 8))

        # Left: RGB image
        axes[0].imshow(rgb_img)
        axes[0].set_title(f"RGB Image (ID: {img_id})", fontsize=12, fontweight='bold')
        axes[0].axis("off")

        # Right: Predicted Yield Map
        im = axes[1].imshow(prediction_map, cmap='jet', vmin=yield_min, vmax=yield_max)
        axes[1].set_title(f"Prediction Map (Band {band_idx})", fontsize=12, fontweight='bold')
        axes[1].axis("off")

        # Display actual yield value
        axes[1].text(0.5, -0.08, f"Yield: {actual_yield:.2f} kg",
                     ha='center', va='top', fontsize=12, fontweight='bold',
                     transform=axes[1].transAxes)

        # Add colorbar
        cbar = fig.colorbar(im, ax=axes[1], orientation='vertical', fraction=0.046, pad=0.04)
        cbar.set_label('Yield (kg)', fontsize=11, fontweight='bold')
        for t in cbar.ax.get_yticklabels():
            t.set_fontweight('bold')

        plt.tight_layout()
        plt.show()

    except FileNotFoundError:
        print(f"❌ File not found for Image ID {img_id}")
    except Exception as e:
        print(f"⚠️ Error for Image ID {img_id}: {e}")

# ==============================================================
# Interactive Widget 
# ==============================================================
interact(
    show_yield_visualization,
    img_id=IntSlider(min=0, max=len(data)-1, step=1, value=0),
    band_idx=IntSlider(min=0, max=len(band_columns)-1, step=1, value=0)
)


In [ ]:
# ===============================================================
# Yield Prediction Visualization (3x3 layout — RGB | Prediction | Colorbar)
# ===============================================================

import numpy as np
import pandas as pd
import tifffile as tiff
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

# ---------------------------------------------------------------
# Load CSV
# ---------------------------------------------------------------
csv_path = r"D:/Z Education/University/4-1 Course/Thesis/Hyper leaf dataset/normalized_minmax_with_target_2.csv"
data = pd.read_csv(csv_path)

# Identify spectral bands
band_columns = [c for c in data.columns if 'nm' in c.lower()]
wavelengths = np.array([float(c.lower().replace('nm', '')) for c in band_columns])

# Identify yield column
yield_col = next((c for c in data.columns if 'yield' in c.lower()), None)
if yield_col:
    yield_min, yield_max = data[yield_col].min(), data[yield_col].max()
else:
    yield_min, yield_max = 0, 1

# ---------------------------------------------------------------
# Visualization Function
# ---------------------------------------------------------------
def show_yield_visualization(start_id, band_idx):
    fig, axes = plt.subplots(3, 2, figsize=(12, 4))
    
    im = None  # To store the last prediction image for the colorbar

    for row, img_id in enumerate(range(start_id, start_id + 3)):
        file_path = f"D:/Z Education/University/4-1 Course/Thesis/Hyper leaf dataset/image/{img_id:05d}.tiff"

        try:
            cube = tiff.imread(file_path)
            if cube.ndim != 3:
                print(f"⚠️ Invalid cube for ID {img_id}")
                continue

            bands, rows, cols = cube.shape
            cube = np.transpose(cube, (1, 2, 0))

            def find_band(target):
                return np.argmin(np.abs(wavelengths - target))

            r_idx, g_idx, b_idx = find_band(660), find_band(550), find_band(470)

            def norm_band(band):
                b = cube[:, :, band]
                return (b - b.min()) / (b.max() - b.min() + 1e-8)

            rgb = np.stack([norm_band(r_idx), norm_band(g_idx), norm_band(b_idx)], axis=-1)

            pred = cube[:, :, band_idx]
            pred = (pred - pred.min()) / (pred.max() - pred.min() + 1e-8)
            pred = pred * (yield_max - yield_min) + yield_min

            actual_yield = data[yield_col].iloc[img_id] if yield_col else np.nan

            # --- RGB image ---
            ax_rgb = axes[row, 0]
            ax_rgb.imshow(rgb)
            if row == 0:
                ax_rgb.set_title("RGB image", fontsize=15, fontweight="bold", y=1.2)  # moved slightly up
            ax_rgb.axis("off")

            # --- Prediction map ---

            ax_pred = axes[row, 1]
            im = ax_pred.imshow(pred, cmap="jet", vmin=yield_min, vmax=yield_max)
            if row == 0:
                ax_pred.set_title("Prediction map", fontsize=15, fontweight="bold", y=1.2)  # moved slightly up
            ax_pred.axis("off")

            # --- Add actual yield below each prediction map ---
            ax_pred.text(
                0.5, -0.15,  # adjust y to move text below the image
                f"Yield: {actual_yield:.2f} kg",
                transform=ax_pred.transAxes,
                ha="center", va="top",
                fontsize=13, fontweight="bold"
            )

            

        except FileNotFoundError:
            print(f"❌ File not found for Image ID {img_id}")
        except Exception as e:
            print(f"⚠️ Error for Image ID {img_id}: {e}")

    # --- Adjust colorbar to a separate Axes ---
    cbar_ax = fig.add_axes([0.92, 0.06, 0.02, 0.83])  # [left, bottom, width, height]
    cbar = fig.colorbar(im, cax=cbar_ax)
    cbar.set_label("Yield (kg)", fontsize=13, fontweight="bold",x=1.5)
    for t in cbar.ax.get_yticklabels():
        t.set_fontweight("bold")

    # --- Tight layout for all images ---
    plt.subplots_adjust(left=0.05, right=0.90, top=0.95, bottom=0.001, hspace=0.15, wspace=0.05)
    plt.show()

# ---------------------------------------------------------------
# Interactive Widget (scroll through images)
# ---------------------------------------------------------------
interact(
    show_yield_visualization,
    start_id=IntSlider(min=0, max=len(data)-4, step=1, value=0),
    band_idx=IntSlider(min=0, max=len(band_columns)-1, step=1, value=0)
)


2.Exceptional Images

In [ ]:
import pandas as pd                       # pandas: for reading CSV files and data manipulation
import numpy as np                        # numpy: for numerical operations and arrays
import matplotlib.pyplot as plt           # matplotlib: for plotting data

# Step 1: Load dataset
file_path = r"D:/Z Education/University/4-1 Course/Thesis/Hyper leaf dataset/normalized_minmax_with_target_2.csv"
                                          # Full path to your CSV file (raw string so backslashes work)
df = pd.read_csv(file_path)               # Read the CSV file into a pandas DataFrame

# Drop target column if it exists
if 'Yield' in df.columns:
    feature_df = df.drop(columns=['Yield'])  # If 'Yield' column exists, drop it (keep only features)
else:
    feature_df = df                         # Otherwise, keep the whole DataFrame as features

num_bands = feature_df.shape[1]             # Number of spectral bands (columns) per row

# Step 2: Identify exceptional rows
min_run_length = 20                   # Condition: ≥25 consecutive values of 0 or 1
exceptional_rows = []                       # Empty list to store (row_index, row_data) of exceptional rows

# Loop over each row of the feature matrix
for idx, row in feature_df.iterrows():
    data = row.values.astype(float)         # Convert row into numpy array (float type)

    run_length = 1                          # Counter for length of consecutive sequence
    exceptional = False                     # Flag: will be True if this row is exceptional

    # Loop through the row values (start from index 1 to compare with previous element)
    for i in range(1, len(data)):
        # Only count if the value is exactly 0 or 1
        # and if it matches the previous value (consecutive repetition)
        if data[i] in [0, 1] and data[i] == data[i-1]:
            run_length += 1                 # Increase the consecutive run counter
            if run_length >= min_run_length:
                exceptional = True          # If the run is long enough, mark as exceptional
                break                       # Stop checking further in this row
        else:
            run_length = 1                  # Reset counter if values differ or not 0/1

    # After finishing the loop, if exceptional, save this row
    if exceptional:
        exceptional_rows.append((idx+2, data))  # Store tuple (row_index, row_data)

# Step 3: Plot exceptional rows with row index labels
if exceptional_rows:
    plt.figure(figsize=(12,6))             # Set figure size

    # Plot each exceptional row’s spectral curve
    for idx, data in exceptional_rows:
        plt.plot(range(len(data)), data, label=f'Row {idx}')  # Label with row index

    # Add title and axis labels
    plt.title(f"Spectral Curves of Exceptional Rows (≥{min_run_length} consecutive 0 or 1)")
    plt.xlabel("Band Index")                # x-axis = band number
    plt.ylabel("Normalized Value")          # y-axis = band value
    plt.grid(True)                          # Add grid for readability
    plt.legend()                            # Show legend (row indexes)
    plt.show()                              # Display the plot

    # Step 4: Print details
    print(f"Total Exceptional Rows: {len(exceptional_rows)}")  # Number of exceptional rows
    print("Exceptional Row Indexes:", [idx for idx, _ in exceptional_rows])  # Their indexes
else:
    print("No exceptional rows found!")     # If no exceptional row detected


In [ ]:
import numpy as np                     # For numerical operations (arrays, math)
import tifffile as tiff                # For reading/writing hyperspectral .tiff images
from skimage import filters            # For image processing (e.g., Laplacian, Gaussian)
from skimage.filters import threshold_otsu  # For automatic Otsu thresholding
from sklearn.cluster import KMeans     # KMeans clustering algorithm
from ipywidgets import interact, IntSlider, IntText  # Interactive widgets for Jupyter
import matplotlib.pyplot as plt        # Plotting library
from matplotlib.colors import ListedColormap  # For discrete cluster color maps
from matplotlib.patches import Patch  # For adding legend boxes

# Define wavelength range
num_bands = 204                        # Total number of spectral bands
wavelength_start = 400
wavelength_end = 1000
wavelengths = np.linspace(wavelength_start, wavelength_end, num_bands)  # Wavelength array

# Function to calculate band metrics
def calculate_band_metrics(band_array):
    band_norm = (band_array - np.nanmin(band_array)) / (np.nanmax(band_array) - np.nanmin(band_array) + 1e-8)
    
    brightness = np.nanmean(band_norm)
    contrast = np.nanstd(band_norm)
    sharpness = filters.laplace(band_norm).var()
    
    highpass = band_norm - filters.gaussian(band_norm, sigma=1)
    noise = np.nanstd(highpass)
    signal = np.sqrt(np.nanmean(band_norm**2))
    snr = 20 * np.log10(signal / (noise + 1e-8))
    
    return {"Brightness": brightness, "Contrast": contrast, "Sharpness": sharpness, "SNR(dB)": snr}

# Interactive function
def show_band_metrics_interactive(img_id, band_idx):
    file_path = f"D:/Z Education/University/4-1 Course/Thesis/Hyper leaf dataset/image/{img_id:05d}.tiff"

    try:
        cube = tiff.imread(file_path)
        bands, rows, cols = cube.shape

        print(f"\nLoaded Image ID: {img_id}")
        print(f"Image width: {cols}, height: {rows}, bands: {bands}, dtype: {cube.dtype}")
        bit_depth = cube.dtype.itemsize * 8
        print(f"Bit depth: {bit_depth}\n")

        cube = np.transpose(cube, (1, 2, 0))
        band_array = cube[:, :, band_idx]

        metrics = calculate_band_metrics(band_array)
        if len(wavelengths) == bands:
            band_wavelength = wavelengths[band_idx]
            print(f"Selected Band: {band_idx} | Wavelength: {band_wavelength:.2f} nm")
        print(f"Brightness: {metrics['Brightness']:.4f}")
        print(f"Contrast: {metrics['Contrast']:.4f}")
        print(f"Sharpness: {metrics['Sharpness']:.4f}")
        print(f"SNR(dB): {metrics['SNR(dB)']:.4f}")

        # Otsu threshold segmentation
        thresh = threshold_otsu(band_array)
        mask = band_array > thresh
        segmented_leaf = band_array * mask

        # KMeans clustering
        n_clusters = 3
        disease_mask = np.zeros_like(band_array)
        if np.sum(mask) > 10:
            leaf_pixels_band = band_array[mask].reshape(-1, 1)
            kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10).fit(leaf_pixels_band)
            disease_mask[mask] = kmeans.labels_ + 1

        # ===== Visualization (2x2 layout) =====
        fig, axes = plt.subplots(2, 2, figsize=(12, 4))
        axes = axes.ravel()

        # (0) RGB Composite
        def find_band_index(target_wl):
            return np.argmin(np.abs(wavelengths - target_wl))

        r_idx = find_band_index(660)
        g_idx = find_band_index(550)
        b_idx = find_band_index(470)

        rgb_img = np.stack([cube[:, :, r_idx],
                            cube[:, :, g_idx],
                            cube[:, :, b_idx]], axis=-1)
        rgb_img = (rgb_img - np.min(rgb_img)) / (np.max(rgb_img) - np.min(rgb_img) + 1e-8)

        axes[0].imshow(rgb_img)
        axes[0].set_title(f"Original Leaf ({img_id})")  # Added Image ID
        axes[0].axis("off")

        # (1) Otsu binary mask
        axes[1].imshow(mask, cmap='gray')
        axes[1].set_title("Otsu Binary Mask")
        axes[1].axis("off")

        # (2) Segmented leaf
        axes[2].imshow(segmented_leaf, cmap='gray')
        axes[2].set_title("Segmented Leaf (Grayscale)")
        axes[2].axis("off")

        # (3) Leaf clusters
        cluster_cmap = ListedColormap(["black", "green", "orange", "blue"])
        axes[3].imshow(disease_mask, cmap=cluster_cmap, vmin=0, vmax=3)
        axes[3].set_title("Leaf Clusters (Healthy, Mild, Severe)")
        axes[3].axis("off")

        # Cluster legend
        legend_elements = [
            Patch(facecolor="green", edgecolor='k', label="Healthy"),
            Patch(facecolor="orange", edgecolor='k', label="Mild Disease"),
            Patch(facecolor="blue", edgecolor='k', label="Severe Disease")
        ]
        axes[3].legend(handles=legend_elements, loc='upper right', fontsize=8)

        plt.tight_layout()
        plt.show()

    except FileNotFoundError:
        print(f"File for Image ID {img_id} not found.")

# Dataset info
min_id = 0
max_id = 2409
first_img_file = f"D:/Z Education/University/4-1 Course/Thesis/Hyper leaf dataset/image/{min_id:05d}.tiff"
first_cube = tiff.imread(first_img_file)
total_bands = first_cube.shape[0]

# Interactive controls
interact(
    show_band_metrics_interactive,
    img_id=IntText(min=min_id, max=max_id, value=min_id, description='Image ID'),
    band_idx=IntSlider(min=0, max=total_bands-1, step=1, value=0, description='Band')
)


3.Image to xlsx

In [ ]:
# image to xlsx file convert
import numpy as np                     # For numerical operations
import tifffile as tiff                # To read hyperspectral .tiff images
from skimage.filters import threshold_otsu   # For automatic thresholding (segmentation)
import pandas as pd                    # To store and save tabular data (Excel)
import os                              # For file and directory handling

# Dataset info
data_dir = r"D:/Z Education/University/4-1 Course/Thesis/Hyper leaf dataset/image/"  
# Path to dataset folder containing hyperspectral images

all_files = sorted([f for f in os.listdir(data_dir) if f.endswith(".tiff")])  
# List all .tiff files in dataset, sorted by name (00000.tiff → 02409.tiff)

num_bands = 204                        # Number of spectral bands per image
wavelengths = np.linspace(400, 1000, num_bands)  
# Create wavelength array from 400–1000 nm (evenly spaced for 204 bands)

# Yield example (0–2409)
num_images = 2410                      # Total images in dataset
np.random.seed(42)                     # Fix random seed for reproducibility
yield_values = np.round(np.random.uniform(3.0, 5.0, size=num_images), 2)  
# Simulated yield values between 3.0–5.0 (2 decimal places)

yield_dict = {i: yield_values[i] for i in range(num_images)}  
# Map each image ID to a simulated yield value

# Function: Extract Metrics
def extract_metrics(file_path):
    cube = tiff.imread(file_path)             # Read hyperspectral cube (bands, rows, cols)
    cube = np.transpose(cube, (1, 2, 0))      # Rearrange to (rows, cols, bands)

    ref_band = cube[:, :, 50]                 # Select one reference band (index 50 ~ ~520nm)
    otsu_thresh = threshold_otsu(ref_band)    # Compute Otsu threshold for segmentation
    mask = ref_band > otsu_thresh             # Binary mask (leaf pixels = True)

    leaf_pixels = cube[mask, :]               # Extract spectra of only leaf pixels
    mean_spectrum = np.mean(leaf_pixels, axis=0)  # Average reflectance across bands
    min_reflectance = np.min(leaf_pixels)     # Minimum reflectance value (for health check)

    return mean_spectrum, min_reflectance, otsu_thresh  
    # Return metrics for further processing

# Collect Data
records = []                                  # Empty list to store all rows

for file_name in all_files:                   # Loop through every TIFF image
    try:
        img_id = int(file_name.split(".")[0]) # Extract numeric ID from filename
        file_path = os.path.join(data_dir, file_name)  # Build full file path

        mean_spec, min_ref, otsu_val = extract_metrics(file_path)  
        # Extract spectrum, min reflectance & Otsu threshold for this image

        yield_val = yield_dict.get(img_id, np.nan)  
        # Get yield value for this image ID (or NaN if missing)

        # Clear & readable column names
        record = {"Image ID": img_id}         # Start with image ID
        for i, wl in enumerate(wavelengths):  # Loop through all bands
            record[f"{int(wl)} nm"] = mean_spec[i]  
            # Save mean reflectance for each wavelength (e.g., "400 nm", "405 nm"...)

        # Right-most metrics
        record["Minimum Reflectance"] = min_ref   # Store min reflectance value
        record["Otsu Threshold"] = otsu_val       # Store segmentation threshold
        record["Yield"] = yield_val               # Store corresponding yield

        records.append(record)                    # Add this record (row) to list

    except FileNotFoundError:                     # If file not found → skip
        print(f"Image {file_name} not found, skipping...")

# Save to Excel
df = pd.DataFrame(records)                        # Convert all records into DataFrame
output_file = "hyperspectral.xlsx"  # Output Excel file name
df.to_excel(output_file, index=False)             # Save DataFrame to Excel (no row index)

print(f"✅ Excel saved with clean headers: {output_file}")  
# Confirmation message
